# Indeed Webscraper

## Imports

In [4]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

import os
import csv
import logging

import pandas as pd

## Indeed Job Search

In [20]:
loadWindow_Time = 60
sleepTime_betweenJobs = 60
attempts = 3


def startChromeEngine():
    Indeed_Url = "https://de.indeed.com/?r=us"
    ser = Service("/Applications/chromedriver")

    # start chrome driver
    driver = webdriver.Chrome(service=ser)
    driver.get(Indeed_Url)

    # wait for cookies window
    time.sleep(loadWindow_Time)

    return driver


def startSearchEngine(driver, job_title, location):
    # accept cookies
    driver.find_element(By.XPATH, "//*[@id=\"onetrust-accept-btn-handler\"]").click()

    # enter job title
    driver.find_element(By.XPATH, "//*[@id=\"text-input-what\"]").send_keys(job_title)

    # enter location
    driver.find_element(By.XPATH, "//*[@id=\"text-input-where\"]").send_keys(location)
    # start search
    driver.find_element(By.XPATH, "//*[@id=\"text-input-where\"]").send_keys(Keys.RETURN)

    # wait for page to load 
    time.sleep(loadWindow_Time)
    # sort result by publish data
    driver.find_element(By.XPATH, "//*[@id=\"resultsCol\"]/div[3]/div[4]/div[1]/span[2]/a").click()
    # wait for random pop up window
    time.sleep(loadWindow_Time)
    
    # close random pop up window
    driver.find_element(By.XPATH,"//*[@id=\"popover-x\"]/button").click()
    
    time.sleep(loadWindow_Time)

    return driver


def startScrapingPage(driver, all_jobs_lst):
    # get the job list container
    for attempt in range(attempts):
        try:
            job_list_container = driver.find_element(By.XPATH, "//*[@id=\"mosaic-provider-jobcards\"]")
        except:
            time.sleep(10)

            
    # get all jobs in container
    job_list = job_list_container.find_elements(By.TAG_NAME, "a")

    # get all job ids in job_list
    job_ids = []
    try:
        for job in job_list:
            id = job.get_attribute("id")
            if id != "":
                job_ids.append(id)
    except:
        time.sleep(loadWindow_Time)

    # iterate over every job in job_ids
    for job_id in job_ids:
        try:
            driver.find_element(By.XPATH, f"//*[@id=\"{job_id}\"]").click()
            time.sleep(sleepTime_betweenJobs)
        except:
            continue

        # switch to Job Description iframe
        for attempt in range (attempts):
            try:
                iframe = driver.find_element(By.XPATH,"//*[@id=\"vjs-container-iframe\"]")
                driver.switch_to.frame(iframe)
                break
            except:
                time.sleep(loadWindow_Time)

        for attempt in range(attempts):
            # get job top card
            try:
                topCard = driver.find_element(By.XPATH, "//*[@id=\"viewJobSSRRoot\"]/div/div[1]/div/div/div/div[1]/div/div[1]")
            except:
                job_title = ""
                company_name = ""
                company_location = ""
                job_type = ""
                time.sleep(loadWindow_Time)
                continue

            # get job title
            try:
                job_title = topCard.find_element(By.CSS_SELECTOR,".icl-u-xs-mb--xs.icl-u-xs-mt--none.jobsearch-JobInfoHeader-title.is-embedded").text
            except:
                job_title = ""

            # get company name
            try:
                company_name = driver.find_element(By.CSS_SELECTOR, ".icl-u-lg-mr--sm.icl-u-xs-mr--xs").text
            except:
                company_name = ""

            # get company location
            try:
                company_location = driver.find_element(By.CSS_SELECTOR,"div[class='jobsearch-CompanyInfoWithoutHeaderImage'] div:nth-child(2)").text
            except:
                company_location = ""
            # get type of job
            try:
                job_type = driver.find_element(By.CSS_SELECTOR, "body > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > span:nth-child(1)").text
            except:
                job_type = ""


        # get job description
        for attempt in range(attempts):
            try:
                job_Description = driver.find_element(By.XPATH, "//*[@id=\"jobDescriptionText\"]").text
                break
            except:
                job_Description = ""
                time.sleep(loadWindow_Time)

        # get current datetime
        date_time = datetime.datetime.now().strftime("%d%b%Y-%H:%M:%S")

        # add all_jobs to all_jobs_lst
        all_jobs = [job_id, job_title, company_name, company_location, job_type, job_Description, date_time]
        all_jobs_lst.append(all_jobs)

        # switch back to default frame to click on next job
        driver.switch_to.default_content()

    return all_jobs_lst


def getNextPage(driver):
    # click on next Page button
    driver.find_element(By.XPATH,"//*[@id=\"resultsCol\"]/nav/div/ul/li[6]/a").click()
    time.sleep(loadWindow_Time)

    return driver


def saveAsCSV(all_jobs_lst, page_nr):
    dateTime=datetime.datetime.now()    
    filename_dateTime = dateTime.strftime("%d%m%Y_%H_%M_%S")
    df = pd.DataFrame(all_jobs_lst)
    df.to_csv("/Users/jan/Documents/7.Semester/Datenanalyse in der Praxis/SeminarArbeit/Data/Indeed_Jobs" + str(page_nr) + str(filename_dateTime)  + ".csv", index=False)
    

# MAIN

In [14]:
ammount_Of_Pages=35
driver = startChromeEngine()

driver = startSearchEngine(driver, "Data Analyst", "Deutschland")

for page_Number in range(ammount_Of_Pages):
    newPage_Number=page_Number
    all_jobs_lst=[]
    all_jobs_lst = startScrapingPage(driver,all_jobs_lst)
    saveAsCSV(all_jobs_lst, newPage_Number)
    driver = getNextPage(driver)



KeyboardInterrupt: 